In [1]:
import torch
from Dataset.Dataset import get_tokenizer
from Model.Model import SLMModel
from tokenizers import Tokenizer
import yaml

In [2]:
with open("config.yaml") as f:
    config = yaml.safe_load(f)

In [ ]:
MAX_SEQ_LEN = 256
VOCAB_SIZE =10000
EMBEDDING_DIM = 512

tokenizer = get_tokenizer(config["tokenizer_path"],MAX_SEQ_LEN)
tokenizer.no_truncation()
tokenizer.no_padding()

device = torch.device("cuda")
model = SLMModel(VOCAB_SIZE,EMBEDDING_DIM,4,8,64,64,0.1,1024,0.2)
model.load_state_dict(torch.load(config["best_model_state_dict"]))
model = model.to(device)

In [7]:
def generate_text(model: torch.nn.Module, tokenizer: Tokenizer, prompt, max_seq_len, device: torch.device, temperature=1.):
    model.eval()  
    
    tokens = tokenizer.encode(prompt).ids
    tokenized_tokens = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0) 
    
    with torch.no_grad(): 
        for i in range(max_seq_len - len(tokens)):
            
            preds = model(tokenized_tokens)
    
            last_token_logits = preds[0, -1, :] 
            
            probs = torch.softmax(last_token_logits / temperature, dim=-1)
            
            next_token_id = torch.multinomial(probs, 1)
            
            tokenized_tokens = torch.cat([
                tokenized_tokens, 
                next_token_id.unsqueeze(0)
            ], dim=1)
            
            if next_token_id.item() == tokenizer.token_to_id("[END]"):
                print("the generation was stopped due to the [END] token")
                break
            

    generated_tokens = tokenized_tokens.squeeze(0).tolist()
    generated_text = tokenizer.decode(generated_tokens)
    
    return generated_text

In [ ]:
prompt = "[START] " #Insert your prompt
generate_text(model,tokenizer,prompt,MAX_SEQ_LEN,device,0.9)

the generation was stopped due to the [END] token


'The little dragon was afraid of fire because it was so big and frightening . He sighed when he saw a big box in the back yard . It was made of wood and had a shiny lock on it . Suddenly , the dragon saw a bright light coming from inside the box . It flew out and into the house . The dragon paused and looked back . He saw his human , who was playing catch with a ball . The dragon smiled and waved at the human , who waved back . The human smiled back and the dragon hopped in and started to play a game of catch . The dragon was so happy that he started to laugh . He dropped the ball on the ground and jumped up and down . Suddenly , the dragon heard his mom \' s voice . She rushed outside and saw him . She said to his mom , " What are you doing ?" The dragon just sighed and said , " Playing catch is much more fun than catching fire ." His mom laughed again . She knew that the dragon was harmless and it was just trying to be fun . The little dragon smiled , grabbed the ball and ran off to 